## LSTM + Deep CNN Text Classification with Keras

In [1]:
%run Setup.ipynb

[i] Loaded Parameters:
 40000 30 0.2 50 
 dataset/glove/glove.twitter.27B.50d.txt
[i] Importing Modules...


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


[i] Using Keras version 2.1.4
[i] Finished Importing Modules
[i] . Reading from csv file...Done!
[i] Found 30692 unique tokens.
[+] Shape of data tensor: (40000, 30)
[+] Shape of label tensor: (40000, 4)
[+] Number of entries in each category:
[+] Training:
 [ 7762. 10890. 12239.  1109.]
[+] Validation:
 [1882. 2734. 3060.  324.]
[i] Loading GloVe from: dataset/glove/glove.twitter.27B.50d.txt ...Done.
[+] Proceeding with Embedding Matrix...Completed!
Finished running setup.


Load embedding matrix into an `Embedding` layer. Toggle `trainable=False` to prevent the weights from being updated during training.

In [2]:
embedding_layer = Embedding(len(word_index) + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)

### LSTM + Deep CNN Structure
[Reference](https://github.com/richliao/textClassifier), [LTSM](http://colah.github.io/posts/2015-08-Understanding-LSTMs/), [CNN Source](https://blog.keras.io/using-pre-trained-word-embeddings-in-a-keras-model.html) and [Notes](http://www.wildml.com/2015/11/understanding-convolutional-neural-networks-for-nlp/)

Deeper CNN as described in [CNN for Sentence Classification](http://www.aclweb.org/anthology/D14-1181) (Yoon Kim, 2014), multiple filters have been applied. This can be implemented using Keras `Merge` Layer.

In [9]:
sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)

l_lstm1 = Bidirectional(LSTM(4,dropout=0.3,recurrent_dropout=0.3,return_sequences=True))(embedded_sequences)

convs, filter_sizes = [], [3,4,5]
for fsz in filter_sizes:
    l_conv = Conv1D(filters=32,kernel_size=fsz,
                    activation='relu',kernel_regularizer=regularizers.l2(0.01))(l_lstm1)
    convs.append(l_conv)

l_merge = Concatenate(axis=1)(convs)
l_pool1 = MaxPooling1D(2)(l_merge)
l_drop1 = Dropout(0.4)(l_pool1)
l_flat = Flatten()(l_drop1)
l_dense = Dense(16, activation='relu')(l_flat)

preds = Dense(4, activation='softmax')(l_dense)

In [10]:
def get_lr_metric(optimizer):
    def lr(y_true, y_pred):
        return optimizer.lr
    return lr

In [11]:
model = Model(sequence_input, preds)
adadelta = optimizers.Adadelta(lr=1.0, rho=0.95, epsilon=None, decay=0.0)
lr_metric = get_lr_metric(adadelta)
model.compile(loss='categorical_crossentropy',
              optimizer=adadelta,
              metrics=['acc', lr_metric])

In [12]:
def step_cyclic(epoch):
    try:
        l_r, decay = 1.0, 0.0001
        if epoch%33==0:multiplier = 10
        else:multiplier = 1
        rate = float(multiplier * l_r * 1/(1 + decay * epoch))
        #print("Epoch",epoch+1,"- learning_rate",rate)
        return rate
    except Exception as e:
        print("Error in lr_schedule:",str(e))
        return float(1.0)
    
def initial_boost(epoch):
    if epoch==0: return float(6.0)
    else: return float(1.0)
        
tensorboard = callbacks.TensorBoard(log_dir='./logs', histogram_freq=4, batch_size=16, write_grads=True , write_graph=True)
model_checkpoints = callbacks.ModelCheckpoint("checkpoint.h5", monitor='val_loss', verbose=0, save_best_only=True, save_weights_only=False, mode='auto', period=1)
lr_schedule = callbacks.LearningRateScheduler(step_cyclic)

In [13]:
model.summary()
print("Training Progress:")
model.fit(x_train, y_train, validation_data=(x_val, y_val),
          epochs=200, batch_size=50,
          callbacks=[tensorboard, model_checkpoints, lr_schedule])

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 30)           0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 30, 50)       1534650     input_3[0][0]                    
__________________________________________________________________________________________________
bidirectional_3 (Bidirectional) (None, 30, 8)        1760        embedding_1[2][0]                
__________________________________________________________________________________________________
conv1d_7 (Conv1D)               (None, 28, 32)       800         bidirectional_3[0][0]            
__________________________________________________________________________________________________
conv1d_8 (

Epoch 31/200
32000/32000 [==============================] - 21s 658us/step - loss: 1.0281 - acc: 0.5549 - lr: 0.9970 - val_loss: 1.0134 - val_acc: 0.5695 - val_lr: 0.9970
Epoch 32/200
32000/32000 [==============================] - 21s 664us/step - loss: 1.0265 - acc: 0.5598 - lr: 0.9969 - val_loss: 1.0163 - val_acc: 0.5686 - val_lr: 0.9969
Epoch 33/200
32000/32000 [==============================] - 21s 660us/step - loss: 1.0254 - acc: 0.5564 - lr: 0.9968 - val_loss: 1.0136 - val_acc: 0.5741 - val_lr: 0.9968
Epoch 34/200
32000/32000 [==============================] - 21s 645us/step - loss: 1.0778 - acc: 0.5305 - lr: 9.9671 - val_loss: 1.0495 - val_acc: 0.5595 - val_lr: 9.9671
Epoch 35/200
32000/32000 [==============================] - 20s 636us/step - loss: 1.0458 - acc: 0.5498 - lr: 0.9966 - val_loss: 1.0232 - val_acc: 0.5698 - val_lr: 0.9966
Epoch 36/200
32000/32000 [==============================] - 20s 630us/step - loss: 1.0395 - acc: 0.5519 - lr: 0.9965 - val_loss: 1.0207 - val_acc

Epoch 79/200
32000/32000 [==============================] - 20s 621us/step - loss: 1.0198 - acc: 0.5592 - lr: 0.9923 - val_loss: 1.0070 - val_acc: 0.5808 - val_lr: 0.9923
Epoch 80/200
32000/32000 [==============================] - 21s 655us/step - loss: 1.0182 - acc: 0.5602 - lr: 0.9922 - val_loss: 1.0069 - val_acc: 0.5801 - val_lr: 0.9922
Epoch 81/200
32000/32000 [==============================] - 21s 643us/step - loss: 1.0193 - acc: 0.5612 - lr: 0.9921 - val_loss: 1.0087 - val_acc: 0.5791 - val_lr: 0.9921


KeyboardInterrupt: 

In [14]:
model.save('ltsm-c.h5')